In [ ]:
%load_ext autoreload

In [ ]:
!ls /data/taoyi/dataset/Lyncam/school/zhuanpan

# 引入必要的库

In [ ]:
%autoreload
import sys,os
import numpy as np
import matplotlib.pyplot as plt
from tianmoucv.isp import fourdirection2xy,poisson_blend
import torch
from tianmoucv.data import TianmoucDataReader_pcie

extra_data = ['/data/taoyi/dataset/Lyncam/2023_02_16_zy/flicker/']
extra_data = ['/data/taoyi/dataset/Lyncam/school/zhuanpan/leinao/2b_10000fps_6klux']
extra_data = ['/data/taoyi/dataset/Lyncam/driving_dataset/20230522/']
extra_data = ['/data/taoyi/dataset/Lyncam/driving_dataset/20230523/']
extra_data = ['/data/taoyi/dataset/Lyncam/school/zhuanpan/leinao/4b_3030fps_6klux_th1/']
extra_data = ['/data/taoyi/dataset/Lyncam/school/zhuanpan/leinao/8b_1515fps_6klux_th0']
extra_data = ['/data/taoyi/dataset/Lyncam/school/zhuanpan']
extra_data = ['/data/taoyi/dataset/Lyncam/school/zhuanpan/leinao/8b_750fps_6klux_th0']
import torch.nn.functional as F
import cv2


key = 'openroad1'
key = "openroad4"
key = "3000rpm"
key = "200rpm"
key = 'flicker2_90cop'
key = '2b_10000fps_230925_3000rpm'
key = "300rpm"

speedUpRate = 1
dataset = TianmoucDataReader_pcie(extra_data,MAXLEN=500*speedUpRate,ifSaveFileDict = False,rod_adc_bit=8,cameraMode=0,
                              matchkey=key,speedUpRate=speedUpRate)

dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1,\
                                          num_workers=4, pin_memory=False, drop_last = False)

count = 0
torchmean = 0

# 融合图像

In [ ]:
%autoreload
import torch.nn as nn
import math
from tianmoucv.reconstructor.integration import grayReconstructor



MSE_LossFn = nn.MSELoss()
speedUpRate = 1


def images_to_video(frame_list,name,size=(640,320),Flip=False):
    fps = 25        
    ftmax = max([np.max(ft) for ft in frame_list])
    ftmin = min([np.min(ft) for ft in frame_list])
    out = cv2.VideoWriter(name,0x7634706d , fps, size)
    for ft in frame_list:
        ft = (ft-ftmin)/(ftmax-ftmin)
        ft2 = (ft*255).astype(np.uint8)
        out.write(ft2)
    out.release()

psnrcount =0
count = 0

key_list = ["300rpm"]

for key in key_list:
    dataset = TianmoucDataReader_pcie(extra_data,MAXLEN=500*speedUpRate,ifSaveFileDict = False,rod_adc_bit=8,cameraMode=0,
                              matchkey=key,speedUpRate=speedUpRate)
    dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1,\
                                          num_workers=4, pin_memory=False, drop_last = False)

    PSNR = 0
    img_list = []
    for index,sample in enumerate(dataLoader,0):
        if index<0:
            continue
        if index<= 20:
            psnrcount += 1
            F0 = sample['F0_HDR'][0,...]
            F1 = sample['F1_HDR'][0,...]
            gray1 = torch.mean(F1,dim=-1)
            gray0 = torch.mean(F0,dim=-1)

            for t in range(sample['rawDiff'].size(2)):

                tsdiff = sample['rawDiff'][0,...]/255.0
                hdr = grayReconstructor(tsdiff,F0,F1,t,TDnoise=0,threshGate=0/255.0)
                hdr = hdr.numpy()
                hdr_show = np.stack([hdr]*3,axis=-1)                
                SD = tsdiff[1:,t,...].permute(1,2,0)
                Ix,Iy= fourdirection2xy(SD)
                gray = poisson_blend(-Ix,-Iy,iteration=20)
                gray = F.interpolate(torch.Tensor(gray).unsqueeze(0).unsqueeze(0), size=(320,640), mode='bilinear').squeeze(0).squeeze(0)
        
                if t==12:
                    plt.figure(figsize=(18,6))
                    plt.subplot(1,3,1)  
                    plt.imshow(gray0,cmap='gray')
                    plt.subplot(1,3,2)  
                    plt.imshow(hdr_show)
                    plt.subplot(1,3,3)  
                    plt.imshow(gray,cmap='gray')
                    plt.show()
                    
                if t==25:
                    MSE_val = MSE_LossFn(torch.Tensor(hdr), gray1) 
                    PSNR += (10 * math.log10(1 / MSE_val.item()))
                    print("PSNR:",PSNR/psnrcount,'dB')
                img_list.append(hdr_show)
        else:
            break
    
    images_to_video(img_list,'../rebuttal/reconstruction/data/result/Direct_'+key+'.mp4')

In [ ]:
images_to_video(img_list,'../rebuttal/reconstruction/data/result/Direct_'+key+'.mp4')